Подготовка модели 

In [78]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import confusion_matrix, roc_auc_score, f1_score, precision_score, recall_score, accuracy_score
import matplotlib.pyplot as plt 
%matplotlib inline 

In [79]:
def pickle_load(file_name):
    with open(file_name, 'rb') as f:
        data = pickle.load(f)
    return data

def pickle_dump(obj, file_name):
    with open(file_name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
def cutoff_metrics(y_valid, predict_proba, rv, cutoff):
    
    for cut in cutoff:    
        cm = confusion_matrix(y_valid, predict_proba > cut)
        FP, FN = cm[0,1], cm[1,0]
        
        print("f1: {:.3f}\trecall: {:.3f}\tprecision: {:.3f}\tacc: {:.3f}\tcutoff: {:.3f} \ttop: {:.3f} \trevenue: {:.1f}".format(
            f1_score(y_valid, predict_proba > cut), 
            recall_score(y_valid, predict_proba > cut), 
            precision_score(y_valid, predict_proba > cut), 
            accuracy_score(y_valid, predict_proba>cut),
            cut, 
            pred_proba[pred_proba > cut].shape[0] / pred_proba.shape[0], 
            rv(FN, FP)
        ))

In [80]:
model = pickle_load('./model/model')
X_train = pickle_load('./model/X_train')
X_valid = pickle_load('./model/X_valid')
y_train = pickle_load('./model/y_train')
y_valid = pickle_load('./model/y_valid')

model.fit(X_train, y_train)
pred_proba = model.predict_proba(X_valid)[:, 1]

Постройте простую экономическую модель для оценки эффекта от внедрения полученного решения на практике.

Например, введите следующие параметры:

    сколько денег в среднем приносит один пользователь в месяц;
    сколько денег в среднем вы будете вкладывать в удержание одного пользователя;
    с какой вероятностью пользователь примет ваше предложение;
    сколько пользователей (например, топ 1% или топ 25% согласно ранжированию по вашей модели) будет участвовать в кампании.

1. Введите еще несколько параметров и на их основе постройте простую экономическую модель: формулу по которой в зависимости от значения исходных параметров вы сможете рассчитать экономический эффект от проведения кампании (= сколько денег вы получили (или наоборот потеряли) от проведения кампании по удержанию) с использование вашей модели.

Представим в качестве кейса небольшую компанию в сфере телекоммуникаций. Используем следующие вводные:

* число клиентов на текущий момент - **1 млн. человек**
* доля оттока клиентов, в месяц - **7%** (априорное значение из тренировочной выборки, рынок динамичен), 
* средний доход с одного клиента, в месяц: **500 руб.**


Т.к. терять прибыль не хочется, то была разработана кампания по удержанию клиентов (набор специальных предложений).
Считаем, что стоимость разработки этой компании экономической оценки предсказательной модели не касается.
Параметры компании следующие:
* средняя стоимость удержания клиента, в месяц: **50 руб.**
* вероятность принятия предложения клиента, "склонного к оттоку": **20%**
* вероятность принятия предложения лояльного клиента: **30%**

Предположим, что данные о вероятностях получены из исторических данных по прошлым предложения.


Таким образом, прибыль от удержания клиентов будем рассчитывать по следующей формуле:

<center>$Revenue=N*P(churn)*P(accept|churn)*(Income - Cost) - N*P(\neg churn)*P(accept|\neg churn)*Cost$</center> где:

$N$ - общее число клиентов

$P(churn)$ - вероятность оттока

$P(accept|churn)$ - вероятность принятия предложения клиента, "склонного к оттоку"

$Income$ - доход с одного клиента

$Cost$ - стоимость удержания одного клиента

Понятно, что если предлагать скидки всем клиентам без исключения, то это не принесет прибыли.
Поэтому задача модели будет состоять в том, чтобы максимально точно определить "цели" предложения среди клиентов.

С учетом предсказательной модели, части формулы выше нужно переписать так:

<center>$N*P(churn) = N*P(churn) - FN$</center>

<center>$N*P(\neg churn) = FP$</center>

<center>$Revenue=(N*P(churn) - FN)*P(accept|churn)*(Income-Cost) - FP*P(accept|\neg churn)*Cost$</center>

При выборе топа будем руководствоваться именно этой формулой прибыли.

In [81]:
income = 500
cost = 50
accept_c = 0.2
accept_nc = 0.3
n = X_valid.shape[0]
p_churn = y_valid.sum() / n

def revenue(FN, FP):
    return (n * p_churn - FN)*accept_c*(income-cost) - FP * accept_nc * cost


2.Исходя из введенных параметров проанализируйте, какой топ пользователей, склонных к оттоку согласно модели, оптимально выбрать для проведения кампании по удержанию для таким образом, чтобы экономический эффект был наибольшим? 

In [82]:
cutoff_metrics(y_valid, pred_proba, revenue, np.linspace(0.05, 0.2, 16))

f1: 0.241	recall: 0.640	precision: 0.149	acc: 0.701	cutoff: 0.050 	top: 0.320 	revenue: 990.0
f1: 0.255	recall: 0.556	precision: 0.165	acc: 0.758	cutoff: 0.060 	top: 0.250 	revenue: 2970.0
f1: 0.272	recall: 0.495	precision: 0.188	acc: 0.803	cutoff: 0.070 	top: 0.196 	revenue: 4620.0
f1: 0.280	recall: 0.427	precision: 0.209	acc: 0.837	cutoff: 0.080 	top: 0.152 	revenue: 5265.0
f1: 0.277	recall: 0.363	precision: 0.224	acc: 0.859	cutoff: 0.090 	top: 0.121 	revenue: 5130.0
f1: 0.273	recall: 0.323	precision: 0.237	acc: 0.872	cutoff: 0.100 	top: 0.101 	revenue: 4995.0
f1: 0.275	recall: 0.293	precision: 0.260	acc: 0.885	cutoff: 0.110 	top: 0.084 	revenue: 5145.0
f1: 0.258	recall: 0.250	precision: 0.266	acc: 0.893	cutoff: 0.120 	top: 0.070 	revenue: 4530.0
f1: 0.231	recall: 0.204	precision: 0.267	acc: 0.899	cutoff: 0.130 	top: 0.057 	revenue: 3705.0
f1: 0.223	recall: 0.185	precision: 0.278	acc: 0.904	cutoff: 0.140 	top: 0.050 	revenue: 3525.0
f1: 0.205	recall: 0.161	precision: 0.283	acc: 0.907

На валидационной выборке (5000 человек) с заданными параметрами наилучшим оказался порог 0.08, или топ размером в **15%** клиентов.
Интересно, что он совпал с оптимальным значением f1-меры. 
Прибыль составляет 5265 рублей на 5000 клиентов или 1.053.000 рублей на всю предполагаемую выборку.

3.Попробуйте усложнить экономическую модель. Добавьте еще несколько параметров и поиграйте с ними (например, измените стоимость удержания и вероятность, с которой пользователь принимает предложение), проанализируйте как меняется оптимальный размер топа? 

Добавим в модель совокупную стоимость затрат на модель и исследование, а также прибыль от подписок на новые сервисы или тарифы, полученные благодаря новому предложению (сопутствующие покупки).

Предположим, что стоимость проекта и исследования составили 200т. рублей.
Тогда для нашей валидационной выборки эта сумма поделится на 200 (1000 т./5т.) и составит 1т. 

In [37]:
model_cost = 1000
income_similar = 20

def revenue(FN, FP):
    return (n * p_churn - FN)*accept_c*(income - cost + income_similar) - FP * accept_nc * cost - model_cost

Ниже примеры топов при разных параметрах.

In [39]:
income, cost, accept_c, accept_nc = 500, 50, 0.2, 0.3
cutoff_metrics(y_valid, pred_proba, revenue, np.linspace(0.05, 0.2, 16))

f1: 0.241	recall: 0.640	precision: 0.149	acc: 0.701	cutoff: 0.050 	top: 0.320 	revenue: 942.0
f1: 0.255	recall: 0.556	precision: 0.165	acc: 0.758	cutoff: 0.060 	top: 0.250 	revenue: 2798.0
f1: 0.272	recall: 0.495	precision: 0.188	acc: 0.803	cutoff: 0.070 	top: 0.196 	revenue: 4356.0
f1: 0.280	recall: 0.427	precision: 0.209	acc: 0.837	cutoff: 0.080 	top: 0.152 	revenue: 4901.0
f1: 0.277	recall: 0.363	precision: 0.224	acc: 0.859	cutoff: 0.090 	top: 0.121 	revenue: 4670.0
f1: 0.273	recall: 0.323	precision: 0.237	acc: 0.872	cutoff: 0.100 	top: 0.101 	revenue: 4475.0
f1: 0.275	recall: 0.293	precision: 0.260	acc: 0.885	cutoff: 0.110 	top: 0.084 	revenue: 4581.0
f1: 0.258	recall: 0.250	precision: 0.266	acc: 0.893	cutoff: 0.120 	top: 0.070 	revenue: 3902.0
f1: 0.231	recall: 0.204	precision: 0.267	acc: 0.899	cutoff: 0.130 	top: 0.057 	revenue: 3009.0
f1: 0.223	recall: 0.185	precision: 0.278	acc: 0.904	cutoff: 0.140 	top: 0.050 	revenue: 2801.0
f1: 0.205	recall: 0.161	precision: 0.283	acc: 0.907

оптимальный топ при добавлении новых параметров не изменился

In [40]:
income, cost, accept_c, accept_nc = 500, 75, 0.2, 0.3
cutoff_metrics(y_valid, pred_proba, revenue, np.linspace(0.05, 0.2, 16))

f1: 0.241	recall: 0.640	precision: 0.149	acc: 0.701	cutoff: 0.050 	top: 0.320 	revenue: -10463.0
f1: 0.255	recall: 0.556	precision: 0.165	acc: 0.758	cutoff: 0.060 	top: 0.250 	revenue: -6067.0
f1: 0.272	recall: 0.495	precision: 0.188	acc: 0.803	cutoff: 0.070 	top: 0.196 	revenue: -2534.0
f1: 0.280	recall: 0.427	precision: 0.209	acc: 0.837	cutoff: 0.080 	top: 0.152 	revenue: -416.5
f1: 0.277	recall: 0.363	precision: 0.224	acc: 0.859	cutoff: 0.090 	top: 0.121 	revenue: 485.0
f1: 0.273	recall: 0.323	precision: 0.237	acc: 0.872	cutoff: 0.100 	top: 0.101 	revenue: 972.5
f1: 0.275	recall: 0.293	precision: 0.260	acc: 0.885	cutoff: 0.110 	top: 0.084 	revenue: 1703.5
f1: 0.258	recall: 0.250	precision: 0.266	acc: 0.893	cutoff: 0.120 	top: 0.070 	revenue: 1517.0
f1: 0.231	recall: 0.204	precision: 0.267	acc: 0.899	cutoff: 0.130 	top: 0.057 	revenue: 1061.5
f1: 0.223	recall: 0.185	precision: 0.278	acc: 0.904	cutoff: 0.140 	top: 0.050 	revenue: 1113.5
f1: 0.205	recall: 0.161	precision: 0.283	acc: 0.

Оптимальный топ при повышении затрат на удержание до 75 рублей составил **8%** - цена ошибки выросла.

In [57]:
income, cost, accept_c, accept_nc, income_similar = 500, 50, 0.2, 0.6, 30
cutoff_metrics(y_valid, pred_proba, revenue, np.linspace(0.05, 0.2, 16))

f1: 0.241	recall: 0.640	precision: 0.149	acc: 0.701	cutoff: 0.050 	top: 0.320 	revenue: -19012.0
f1: 0.255	recall: 0.556	precision: 0.165	acc: 0.758	cutoff: 0.060 	top: 0.250 	revenue: -12448.0
f1: 0.272	recall: 0.495	precision: 0.188	acc: 0.803	cutoff: 0.070 	top: 0.196 	revenue: -7216.0
f1: 0.280	recall: 0.427	precision: 0.209	acc: 0.837	cutoff: 0.080 	top: 0.152 	revenue: -3826.0
f1: 0.277	recall: 0.363	precision: 0.224	acc: 0.859	cutoff: 0.090 	top: 0.121 	revenue: -2080.0
f1: 0.273	recall: 0.323	precision: 0.237	acc: 0.872	cutoff: 0.100 	top: 0.101 	revenue: -1090.0
f1: 0.275	recall: 0.293	precision: 0.260	acc: 0.885	cutoff: 0.110 	top: 0.084 	revenue: 134.0
f1: 0.258	recall: 0.250	precision: 0.266	acc: 0.893	cutoff: 0.120 	top: 0.070 	revenue: 248.0
f1: 0.231	recall: 0.204	precision: 0.267	acc: 0.899	cutoff: 0.130 	top: 0.057 	revenue: 26.0
f1: 0.223	recall: 0.185	precision: 0.278	acc: 0.904	cutoff: 0.140 	top: 0.050 	revenue: 254.0
f1: 0.205	recall: 0.161	precision: 0.283	acc: 0

Предположим что предложение настолько соблазнительно, что вероятность его принять для лояльных клиентов повысилась до 60%, а прибыль от сопутствующих сервисов выросла до 30 рублей на клиента.
Тогда выгоднее взять топ в **3%**, т.к. нужно уменьшит число FP.

4.Всегда ли применение модели экономически оправданно? Приведите пример набора значений параметров, при которых применение модели перестает быть оправданным.

Нет, например, в случае, если предложение недостаточно привелкательно, затраты на удержание высоки или модель низкого качества.

In [62]:
income, cost, accept_c, accept_nc, income_similar = 500, 60, 0.1, 0.3, 20
cutoff_metrics(y_valid, pred_proba, revenue, np.linspace(0.05, 0.2, 16))

f1: 0.241	recall: 0.640	precision: 0.149	acc: 0.701	cutoff: 0.050 	top: 0.320 	revenue: -14568.0
f1: 0.255	recall: 0.556	precision: 0.165	acc: 0.758	cutoff: 0.060 	top: 0.250 	revenue: -10270.0
f1: 0.272	recall: 0.495	precision: 0.188	acc: 0.803	cutoff: 0.070 	top: 0.196 	revenue: -6864.0
f1: 0.280	recall: 0.427	precision: 0.209	acc: 0.837	cutoff: 0.080 	top: 0.152 	revenue: -4540.0
f1: 0.277	recall: 0.363	precision: 0.224	acc: 0.859	cutoff: 0.090 	top: 0.121 	revenue: -3214.0
f1: 0.273	recall: 0.323	precision: 0.237	acc: 0.872	cutoff: 0.100 	top: 0.101 	revenue: -2446.0
f1: 0.275	recall: 0.293	precision: 0.260	acc: 0.885	cutoff: 0.110 	top: 0.084 	revenue: -1584.0
f1: 0.258	recall: 0.250	precision: 0.266	acc: 0.893	cutoff: 0.120 	top: 0.070 	revenue: -1330.0
f1: 0.231	recall: 0.204	precision: 0.267	acc: 0.899	cutoff: 0.130 	top: 0.057 	revenue: -1266.0
f1: 0.223	recall: 0.185	precision: 0.278	acc: 0.904	cutoff: 0.140 	top: 0.050 	revenue: -1048.0
f1: 0.205	recall: 0.161	precision: 0.2

5.Оцените изменение экономического эффекта от проведения кампании по удержанию при увеличении качества модели на 1%? На 3%? При ответе на вопрос укажите, по какой метрике вы оцениваете качество.


In [95]:
def recall(TP, FN):
    return TP/(TP+FN)
def precision(TP, FP):
    return TP/(TP+FP)
def f1(recall, precision):
    return 2*recall*precision/(recall+precision)


def cutoff_metrics(y_valid, predict_proba, rv, cutoff):
    
    for cut in cutoff:    
        cm = confusion_matrix(y_valid, predict_proba > cut)
        FP, FN, TP = cm[0,1], cm[1,0], cm[1,1]
        
        print("f1: {:.3f}\trecall: {:.3f}\tprecision: {:.3f}\tacc: {:.3f}\tcutoff: {:.3f} \ttop: {:.3f} \trevenue: {:.1f}".format(
            f1_score(y_valid, predict_proba > cut), 
            recall_score(y_valid, predict_proba > cut), 
            precision_score(y_valid, predict_proba > cut), 
            accuracy_score(y_valid, predict_proba>cut),
            cut, 
            pred_proba[pred_proba > cut].shape[0] / pred_proba.shape[0], 
            rv(FN, FP)
        ))
        print(FP, FN, TP)

income, cost, accept_c, accept_nc = 500, 50, 0.2, 0.3
cutoff_metrics(y_valid, pred_proba, revenue, [0.08])


f1: 0.280	recall: 0.427	precision: 0.209	acc: 0.837	cutoff: 0.080 	top: 0.152 	revenue: 5265.0
603 213 159


In [107]:
# default values 
FP, FN, TP = 603, 213, 159

for mult in np.linspace(0.8, 1, 10):
    rc, pr = recall(TP, FN*mult), precision(TP, FP*mult)
    print("{:.3f} {:.3f} {:.3f} {}".format(rc, pr, f1(rc, pr), int(revenue(FN*mult, FP*mult))))


0.483 0.248 0.328 10907
0.476 0.243 0.322 10280
0.469 0.238 0.316 9653
0.463 0.233 0.310 9026
0.456 0.229 0.305 8399
0.450 0.224 0.300 7772
0.444 0.220 0.295 7145
0.439 0.216 0.290 6518
0.433 0.212 0.285 5891
0.427 0.209 0.280 5264


In [106]:
(6518-5264)*200,  (9026-5264)*200

(250800, 752400)

При повышении f1 меры на 1% - 250т.р., при повышении f1 меры на 3% - 750т.р.

6.Как вы думаете, является ли экономически оправданным вложение средств в улучшение качества модели? На сколько нужно улучшить модель, чтобы это качественно сказалось на экономическом эффекте от удержания?

Думаю, что повышение качества модели на заданных параметрах оправданно.